In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

model = torch.load(MODEL_PATH, map_location=device,weights_only=False)
model.eval()

print("✅ Scratch model loaded successfully")


✅ Scratch model loaded successfully


In [ ]:
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])


In [ ]:
SMOOTHING_FRAMES = 5
face_histories = {}


In [ ]:
print("🎥 Webcam started — press Q to quit")
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=6,
        minSize=(100, 100)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2)

    current_face_ids = set()

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        # 🔑 Face ID (coarse grid tracking)
        face_id = (x // 50, y // 50)
        current_face_ids.add(face_id)

        # Preprocess
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        # Inference
        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        label = classes[pred.item()]
        confidence = conf.item()

        # Init history buffer
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(label)

        # Majority vote
        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # Color
        if stable_label == "Mask":
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)

        # Draw
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        cv2.rectangle(frame, (x, y-35), (x+w, y), color, -1)
        cv2.putText(frame,
                    f"{stable_label} ({confidence*100:.1f}%)",
                    (x+5, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 255),
                    2)

    # Clean old faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Scratch Mask Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# =========================
# IMPORTS
# =========================
import sys
import os
sys.path.append(os.path.abspath(".."))
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

model = torch.load(MODEL_PATH, map_location=device,weights_only=False)
model.eval()

print("✅ Scratch model loaded successfully")

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

if face_cascade.empty():
    raise RuntimeError("❌ Failed to load Haar Cascade")

print("✅ Face detector loaded")

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("❌ Could not open webcam")

print("🎥 Webcam started — press Q to quit")

# =========================
# SMOOTHING
# =========================
SMOOTHING_FRAMES = 5
face_histories = {}

# =========================
# INFERENCE LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=6,
        minSize=(100, 100)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2)

    current_face_ids = set()

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        # 🔑 Coarse face ID (prevents flicker)
        face_id = (x // 50, y // 50)
        current_face_ids.add(face_id)

        # Preprocess face
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        # Inference
        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        label = classes[pred.item()]
        confidence = conf.item()

        # Init smoothing buffer
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(label)

        # Majority vote smoothing
        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # Color logic
        if stable_label == "Mask":
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)

        # Draw bounding box + label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        cv2.rectangle(frame, (x, y-35), (x+w, y), color, -1)
        cv2.putText(
            frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 255, 255),
            2
        )

    # Cleanup old faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Scratch Mask Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# =========================
# CLEANUP
# =========================
cap.release()
cv2.destroyAllWindows()


Using device: cuda
✅ Scratch model loaded successfully
✅ Face detector loaded
🎥 Webcam started — press Q to quit


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque

# =========================
# 🔧 TUNING
# =========================
CONFIDENCE_THRESHOLD = 0.50
SMOOTHING_FRAMES = 7

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL (SAFE)
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

model = torch.load(MODEL_PATH, map_location="cpu",weights_only=False)
model = model.to(device)
model.eval()

print("✅ Scratch model loaded")

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)
print("🎥 Stable Mask Detector Started (Press Q)")

# =========================
# SMOOTHING MEMORY
# =========================
face_histories = {}

# =========================
# INFERENCE LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90, 90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2)

    current_face_ids = set()

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        # Stable face ID
        face_id = (x // 50, y // 50)
        current_face_ids.add(face_id)

        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        raw_label = classes[pred.item()]

        # Confidence guard
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # Init history
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        # Majority vote
        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # Color logic
        if stable_label == "Mask":
            color = (0, 255, 0)
        elif stable_label == "No Mask":
            color = (0, 0, 255)
        else:
            color = (255, 255, 0)

        # Draw
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        cv2.rectangle(frame, (x, y-35), (x+w, y), color, -1)
        cv2.putText(
            frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 255, 255),
            2
        )

    # Cleanup lost faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Scratch Mask Detector (Stable)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Using device: cuda
✅ Scratch model loaded
🎥 Stable Mask Detector Started (Press Q)


In [ ]:
import sys
import os

# go to project root folder
sys.path.append(os.path.abspath(".."))
from mongo_log import save_log
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque

# =========================
# 🔧 TUNING
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.30     # If >30% of the "mask" is skin color, it's likely a hand
SMOOTHING_FRAMES = 7

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL (SAFE)
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Scratch model loaded")
except FileNotFoundError:
    print("❌ ERROR: Model file not found. Check path.")
    exit()

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)
print("🎥 Stable Mask Detector Started (Press Q)")

# =========================
# SMOOTHING MEMORY
# =========================
face_histories = {}

# =========================
# INFERENCE LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90, 90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2)

    current_face_ids = set()

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        # Stable face ID
        face_id = (x // 50, y // 50)
        current_face_ids.add(face_id)

        # -------------------------
        # AI INFERENCE
        # -------------------------
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # -------------------------
        # ✋ HAND GUARD (SKIN CHECK)
        # -------------------------
        # Only run this check if model thinks it is a MASK
        if pred_idx == 0: 
            # 1. Convert ROI to HSV color space (better for skin detection)
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
            
            # 2. Define range for skin color (Generic skin tone)
            lower_skin = np.array([0, 30, 60], dtype=np.uint8)
            upper_skin = np.array([20, 255, 255], dtype=np.uint8)
            
            # 3. Calculate % of skin in the face box
            mask = cv2.inRange(hsv, lower_skin, upper_skin)
            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0] * face_roi.shape[1]
            skin_ratio = skin_pixels / total_pixels

            # 4. If mostly skin (>30%), it's probably a hand, not a mask
            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # -------------------------
        # CONFIDENCE GUARD
        # -------------------------
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # -------------------------
        # SMOOTHING
        # -------------------------
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        # Majority vote
        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # -------------------------
        # DRAWING & COLORS
        # -------------------------
        if stable_label == "Mask":
            color = (0, 255, 0)      # Green
        elif stable_label == "No Mask":
            color = (0, 0, 255)      # Red
        else:
            color = (255, 255, 0)    # Cyan (Uncertain)

        # Draw Box & Label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        cv2.rectangle(frame, (x, y-35), (x+w, y), color, -1)
        cv2.putText(
            frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 255, 255),
            2
        )

    # Cleanup lost faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Scratch Mask Detector (Stable)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Using device: cuda
✅ Scratch model loaded
🎥 Stable Mask Detector Started (Press Q)


In [ ]:
# Media pipe pretrained model google 

import sys
import os

# go to project root folder
sys.path.append(os.path.abspath(".."))
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import mediapipe as mp

# =========================
# 🔧 TUNING
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.60
SMOOTHING_FRAMES = 7

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Scratch model loaded")
except FileNotFoundError:
    print("❌ ERROR: Model file not found. Check path.")
    exit()

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# =========================
# MEDIAPIPE FACE DETECTOR
# =========================
mp_face = mp.solutions.face_detection
face_detector = mp_face.FaceDetection(
    model_selection=0,
    min_detection_confidence=0.5
)

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)
print("🎥 Smart Mask Detector Started (Press Q)")

# =========================
# SMOOTHING MEMORY
# =========================
face_histories = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detector.process(rgb)

    faces = []
    if results.detections:
        h, w, _ = frame.shape
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box

            x = int(bbox.xmin * w)
            y = int(bbox.ymin * h)
            bw = int(bbox.width * w)
            bh = int(bbox.height * h)

            x = max(0, x)
            y = max(0, y)
            bw = min(bw, w - x)
            bh = min(bh, h - y)

            faces.append((x, y, bw, bh))

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2)

    current_face_ids = set()

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x // 50, y // 50)
        current_face_ids.add(face_id)

        # -------------------------
        # AI INFERENCE
        # -------------------------
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # -------------------------
        # ✋ HAND GUARD
        # -------------------------
        if pred_idx == 0:  
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)

            lower_skin = np.array([0, 30, 60], dtype=np.uint8)
            upper_skin = np.array([20, 255, 255], dtype=np.uint8)

            mask = cv2.inRange(hsv, lower_skin, upper_skin)
            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0] * face_roi.shape[1]
            skin_ratio = skin_pixels / total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # -------------------------
        # CONFIDENCE CHECK
        # -------------------------
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # -------------------------
        # SMOOTHING
        # -------------------------
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # -------------------------
        # DRAWING
        # -------------------------
        if stable_label == "Mask":
            color = (0, 255, 0)
        elif stable_label == "No Mask":
            color = (0, 0, 255)
        else:
            color = (255, 255, 0)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        cv2.rectangle(frame, (x, y-35), (x+w, y), color, -1)

        cv2.putText(frame,
                    f"{stable_label} ({confidence*100:.1f}%)",
                    (x+5, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255,255,255),
                    2)

    # cleanup old faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Smart Mask Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
face_detector.close()


Using device: cuda
✅ Scratch model loaded
🎥 Smart Mask Detector Started (Press Q)


In [ ]:
# Media pipe model pretrained google 

import sys
import os

# go to project root folder
sys.path.append(os.path.abspath(".."))
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import mediapipe as mp

# =========================
# 🔧 TUNING
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.60
SMOOTHING_FRAMES = 7

# Ignore fake detections like hand/object
MIN_FACE_SIZE = 120

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Scratch model loaded")
except FileNotFoundError:
    print("❌ ERROR: Model file not found. Check path.")
    exit()

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# =========================
# MEDIAPIPE FACE DETECTOR
# =========================
mp_face = mp.solutions.face_detection
face_detector = mp_face.FaceDetection(
    model_selection=0,
    min_detection_confidence=0.5
)

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)
print("🎥 Smart Mask Detector Started (Press Q)")

face_histories = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detector.process(rgb)

    faces = []
    if results.detections:
        h, w, _ = frame.shape
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box

            x = int(bbox.xmin * w)
            y = int(bbox.ymin * h)
            bw = int(bbox.width * w)
            bh = int(bbox.height * h)

            x = max(0, x)
            y = max(0, y)
            bw = min(bw, w - x)
            bh = min(bh, h - y)

            faces.append((x, y, bw, bh))

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2)

    current_face_ids = set()

    for (x, y, w, h) in faces:

        # 🚫 Ignore small fake detections like hand/object
        if w < MIN_FACE_SIZE or h < MIN_FACE_SIZE:
            continue

        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x // 50, y // 50)
        current_face_ids.add(face_id)

        # ================= AI INFERENCE =================
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # ================= HAND GUARD =================
        if pred_idx == 0:  
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)

            lower_skin = np.array([0, 30, 60], dtype=np.uint8)
            upper_skin = np.array([20, 255, 255], dtype=np.uint8)

            mask = cv2.inRange(hsv, lower_skin, upper_skin)
            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0] * face_roi.shape[1]
            skin_ratio = skin_pixels / total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # ================= CONFIDENCE CHECK =================
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # ================= SMOOTHING =================
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # ================= DRAW =================
        if stable_label == "Mask":
            color = (0, 255, 0)
        elif stable_label == "No Mask":
            color = (0, 0, 255)
        else:
            color = (255, 255, 0)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
        cv2.rectangle(frame, (x, y-35), (x+w, y), color, -1)

        cv2.putText(frame,
                    f"{stable_label} ({confidence*100:.1f}%)",
                    (x+5, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255,255,255),
                    2)

    # cleanup old faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Smart Mask Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
face_detector.close()


Using device: cuda
✅ Scratch model loaded
🎥 Smart Mask Detector Started (Press Q)


In [ ]:
import sys
import os

# go to project root folder
sys.path.append(os.path.abspath(".."))
from mongo_log import save_log
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import time

# =========================
# 🔧 TUNING
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.30
SMOOTHING_FRAMES = 7
LOG_COOLDOWN = 5   # seconds before logging same face again

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Scratch model loaded")
except FileNotFoundError:
    print("❌ ERROR: Model file not found.")
    exit()

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)
print("🎥 Stable Mask Detector Started (Press Q)")

# =========================
# MEMORY
# =========================
face_histories = {}
last_logged_time = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90, 90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,(255,255,0),2)

    current_face_ids = set()

    for (x,y,w,h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x//50, y//50)
        current_face_ids.add(face_id)

        # ======================
        # INFERENCE
        # ======================
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # ======================
        # HAND GUARD
        # ======================
        if pred_idx == 0:
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,30,60])
            upper_skin = np.array([20,255,255])
            mask = cv2.inRange(hsv, lower_skin, upper_skin)

            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0]*face_roi.shape[1]
            skin_ratio = skin_pixels/total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # ======================
        # CONFIDENCE GUARD
        # ======================
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # ======================
        # SMOOTHING
        # ======================
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # ======================
        # 🚀 MONGODB LOGGING
        # ======================
        current_time = time.time()

        if stable_label in ["Mask","No Mask"]:
            if face_id not in last_logged_time:
                last_logged_time[face_id] = 0

            if current_time - last_logged_time[face_id] > LOG_COOLDOWN:
                save_log(stable_label, confidence, frame)
                last_logged_time[face_id] = current_time
                print(f"📝 Logged: {stable_label}")

        # ======================
        # DRAWING
        # ======================
        if stable_label == "Mask":
            color=(0,255,0)
        elif stable_label=="No Mask":
            color=(0,0,255)
        else:
            color=(255,255,0)

        cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)
        cv2.rectangle(frame,(x,y-35),(x+w,y),color,-1)
        cv2.putText(frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5,y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,(255,255,255),2)

    # cleanup
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Mask Detector (Mongo Logging)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from mongo_log import save_log
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import time
import os

# =========================
# 🔧 SETTINGS
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.30
SMOOTHING_FRAMES = 7
LOG_COOLDOWN = 5   # seconds before logging same face again

# create image folder if not exists
if not os.path.exists("logs_images"):
    os.makedirs("logs_images")

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print(" Scratch model loaded")
except FileNotFoundError:
    print(" ERROR: Model not found")
    exit()

# =========================
# CLASSES & TRANSFORM
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# WEBCAM
# =========================
cap = cv2.VideoCapture(0)
print(" Stable Mask Detector Started (Press Q)")

# =========================
# MEMORY
# =========================
face_histories = {}
last_logged_time = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90,90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,(255,255,0),2)

    current_face_ids = set()

    for (x,y,w,h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x//50, y//50)
        current_face_ids.add(face_id)

        # ======================
        # AI INFERENCE
        # ======================
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # ======================
        # HAND GUARD
        # ======================
        if pred_idx == 0:
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,30,60])
            upper_skin = np.array([20,255,255])
            mask = cv2.inRange(hsv, lower_skin, upper_skin)

            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0]*face_roi.shape[1]
            skin_ratio = skin_pixels/total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # ======================
        # CONFIDENCE GUARD
        # ======================
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # ======================
        # SMOOTHING
        # ======================
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # ======================
        # 🚀 LOGGING SYSTEM
        # ======================
        current_time = time.time()

        if stable_label in ["Mask","No Mask"]:
            if face_id not in last_logged_time:
                last_logged_time[face_id] = 0

            if current_time - last_logged_time[face_id] > LOG_COOLDOWN:

                # unique image name
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                img_path = f"logs_images/{timestamp}.jpg"

                # save image
                cv2.imwrite(img_path, frame)

                # save to mongodb + txt
                save_log(stable_label, confidence, img_path)

                last_logged_time[face_id] = current_time
                print(f" Logged: {stable_label}")

        # ======================
        # DRAWING
        # ======================
        if stable_label == "Mask":
            color=(0,255,0)
        elif stable_label=="No Mask":
            color=(0,0,255)
        else:
            color=(255,255,0)

        cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)
        cv2.rectangle(frame,(x,y-35),(x+w,y),color,-1)
        cv2.putText(frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5,y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,(255,255,255),2)

    # cleanup lost faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("Mask Detector (Mongo Logging)", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from mongo_log import save_log
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import time

# =========================
# 👤 CURRENT LOGGED USER (CHANGE HERE)
# =========================
USER_EMAIL = "nikhil@gmail.com"   # admin email
USER_ROLE = "admin"               # admin / user

# later this will come from login token automatically

# =========================
# 🔧 SETTINGS
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.30
SMOOTHING_FRAMES = 7
LOG_COOLDOWN = 5

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Scratch model loaded")
except:
    print("❌ ERROR: Model not found")
    exit()

# =========================
# CLASSES
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# CAMERA
# =========================
cap = cv2.VideoCapture(0)
print("🎥 AI Monitoring Started (Press Q to exit)")

# =========================
# MEMORY
# =========================
face_histories = {}
last_logged_time = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90,90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,(255,255,0),2)

    current_face_ids = set()

    for (x,y,w,h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x//50, y//50)
        current_face_ids.add(face_id)

        # ======================
        # 🤖 AI PREDICTION
        # ======================
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # ======================
        # ✋ HAND GUARD
        # ======================
        if pred_idx == 0:
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,30,60])
            upper_skin = np.array([20,255,255])
            mask = cv2.inRange(hsv, lower_skin, upper_skin)

            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0]*face_roi.shape[1]
            skin_ratio = skin_pixels/total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # ======================
        # CONFIDENCE CHECK
        # ======================
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # ======================
        # SMOOTHING
        # ======================
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # ======================
        # 🚀 LOG TO DATABASE
        # ======================
        current_time = time.time()

        if stable_label in ["Mask","No Mask"]:
            if face_id not in last_logged_time:
                last_logged_time[face_id] = 0

            if current_time - last_logged_time[face_id] > LOG_COOLDOWN:

                save_log(
                    stable_label,
                    confidence,
                    frame,
                    email=USER_EMAIL,
                    role=USER_ROLE,
                )

                last_logged_time[face_id] = current_time
                print(f"📝 Logged: {stable_label} | {USER_EMAIL}")

        # ======================
        # DRAW BOX
        # ======================
        if stable_label == "Mask":
            color=(0,255,0)
        elif stable_label=="No Mask":
            color=(0,0,255)
        else:
            color=(255,255,0)

        cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)
        cv2.rectangle(frame,(x,y-35),(x+w,y),color,-1)
        cv2.putText(frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5,y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,(255,255,255),2)

    # cleanup lost faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("AI Monitoring System", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from mongo_log import save_log
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import time

# =========================
# 👤 CURRENT USER (CHANGE HERE)
# =========================
USER_EMAIL = "nikhil@gmail.com"
USER_ROLE = "admin"   # admin/user

# =========================
# SETTINGS
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.30
SMOOTHING_FRAMES = 7
LOG_COOLDOWN = 5

# =========================
# CREATE IMAGE FOLDER
# =========================
if not os.path.exists("logs_images"):
    os.makedirs("logs_images")

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Model loaded")
except:
    print("❌ Model not found")
    exit()

# =========================
# CLASSES
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# CAMERA
# =========================
cap = cv2.VideoCapture(0)
print("🎥 AI Monitoring Started (Press Q to exit)")

# =========================
# MEMORY
# =========================
face_histories = {}
last_logged_time = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90,90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,(255,255,0),2)

    current_face_ids = set()

    for (x,y,w,h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x//50, y//50)
        current_face_ids.add(face_id)

        # ======================
        # AI PREDICTION
        # ======================
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # ======================
        # HAND GUARD
        # ======================
        if pred_idx == 0:
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,30,60])
            upper_skin = np.array([20,255,255])
            mask = cv2.inRange(hsv, lower_skin, upper_skin)

            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0]*face_roi.shape[1]
            skin_ratio = skin_pixels/total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # ======================
        # CONFIDENCE CHECK
        # ======================
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # ======================
        # SMOOTHING
        # ======================
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # ======================
        # 🚀 SAVE LOG
        # ======================
        current_time = time.time()

        if stable_label in ["Mask","No Mask"]:
            if face_id not in last_logged_time:
                last_logged_time[face_id] = 0

            if current_time - last_logged_time[face_id] > LOG_COOLDOWN:

                # save image first
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                img_path = f"logs_images/{timestamp}.jpg"
                cv2.imwrite(img_path, frame)

                # save mongodb + txt
                save_log(
                    stable_label,
                    confidence,
                    img_path,
                    email=USER_EMAIL,
                    role=USER_ROLE,
                    source="system"   # 👈 IMPORTANT
                )

                last_logged_time[face_id] = current_time
                print(f"📝 Logged: {stable_label} | {USER_EMAIL}")

        # ======================
        # DRAW BOX
        # ======================
        if stable_label == "Mask":
            color=(0,255,0)
        elif stable_label=="No Mask":
            color=(0,0,255)
        else:
            color=(255,255,0)

        cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)
        cv2.rectangle(frame,(x,y-35),(x+w,y),color,-1)
        cv2.putText(frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5,y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,(255,255,255),2)

    # cleanup lost faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("AI Monitoring System", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from mongo_log import save_log
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from collections import deque
import time

# =========================
# 👤 CURRENT USER (CHANGE HERE)
# =========================
USER_EMAIL = "nikhil@gmail.com"
USER_ROLE = "admin"   # admin/user

# =========================
# SETTINGS
# =========================
CONFIDENCE_THRESHOLD = 0.50
SKIN_RATIO_THRESHOLD = 0.30
SMOOTHING_FRAMES = 7
LOG_COOLDOWN = 5

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# LOAD MODEL
# =========================
MODEL_PATH = "../Models/scratch_mask_cnn_best.pth"

try:
    model = torch.load(MODEL_PATH, map_location="cpu", weights_only=False)
    model = model.to(device)
    model.eval()
    print("✅ Model loaded")
except:
    print("❌ Model not found")
    exit()

# =========================
# CLASSES
# =========================
classes = ["Mask", "No Mask"]

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# =========================
# FACE DETECTOR
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# =========================
# CAMERA
# =========================
cap = cv2.VideoCapture(0)
print("🎥 AI Monitoring Started (Press Q to exit)")

# =========================
# MEMORY
# =========================
face_histories = {}
last_logged_time = {}

# =========================
# LOOP
# =========================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(90,90)
    )

    cv2.putText(frame, f"Faces: {len(faces)}",
                (20,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,(255,255,0),2)

    current_face_ids = set()

    for (x,y,w,h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if face_roi.size == 0:
            continue

        face_id = (x//50, y//50)
        current_face_ids.add(face_id)

        # ======================
        # AI PREDICTION
        # ======================
        img = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
        input_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probs = torch.softmax(output, dim=1)[0]
            conf, pred = torch.max(probs, 0)

        confidence = conf.item()
        pred_idx = pred.item()
        raw_label = classes[pred_idx]

        # ======================
        # HAND GUARD
        # ======================
        if pred_idx == 0:
            hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
            lower_skin = np.array([0,30,60])
            upper_skin = np.array([20,255,255])
            mask = cv2.inRange(hsv, lower_skin, upper_skin)

            skin_pixels = cv2.countNonZero(mask)
            total_pixels = face_roi.shape[0]*face_roi.shape[1]
            skin_ratio = skin_pixels/total_pixels

            if skin_ratio > SKIN_RATIO_THRESHOLD:
                raw_label = "UNCERTAIN"

        # ======================
        # CONFIDENCE CHECK
        # ======================
        if confidence < CONFIDENCE_THRESHOLD:
            raw_label = "UNCERTAIN"

        # ======================
        # SMOOTHING
        # ======================
        if face_id not in face_histories:
            face_histories[face_id] = deque(maxlen=SMOOTHING_FRAMES)

        face_histories[face_id].append(raw_label)

        stable_label = max(
            set(face_histories[face_id]),
            key=face_histories[face_id].count
        )

        # ======================
        # 🚀 SAVE LOG
        # ======================
        current_time = time.time()

        if stable_label in ["Mask","No Mask"]:
            if face_id not in last_logged_time:
                last_logged_time[face_id] = 0

            if current_time - last_logged_time[face_id] > LOG_COOLDOWN:

                # save image first
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                img_path = f"logs_images/{timestamp}.jpg"
                cv2.imwrite(img_path, frame)

                # save mongodb + txt
                save_log(
                    stable_label,
                    confidence,
                    frame,
                    email=USER_EMAIL,
                    role=USER_ROLE,
                    source="System"
                )

                last_logged_time[face_id] = current_time
                print(f" Logged: {stable_label} | {USER_EMAIL}")

        # ======================
        # DRAW BOX
        # ======================
        if stable_label == "Mask":
            color=(0,255,0)
        elif stable_label=="No Mask":
            color=(0,0,255)
        else:
            color=(255,255,0)

        cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)
        cv2.rectangle(frame,(x,y-35),(x+w,y),color,-1)
        cv2.putText(frame,
            f"{stable_label} ({confidence*100:.1f}%)",
            (x+5,y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,(255,255,255),2)

    # cleanup lost faces
    for fid in list(face_histories.keys()):
        if fid not in current_face_ids:
            del face_histories[fid]

    cv2.imshow("AI Monitoring System", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
